In [ ]:
def descTidalChar(hilotides,prh,feed,depTides, tagonDS, tagoffDS): 
    # SET AS DT VARS
    feed['sFEDS'] = pd.to_datetime(feed['sFEDS'])
    hilotides['Date Time'] = pd.to_datetime(hilotides['Date Time'])
    depTides['DateTime'] = pd.to_datetime(depTides['DateTime'])
    tagonDS = pd.to_datetime(tagonDS).strftime('%Y-%m-%d %H:%M:%S')
    tagoffDS = pd.to_datetime(tagoffDS).strftime('%Y-%m-%d %H:%M:%S')
    tagonDS = pd.to_datetime(tagonDS)
    tagoffDS = pd.to_datetime(tagoffDS)

    def isinTimeRange(searchdf, starttime, endtime):
        # Convert strings to datetime objects
        starttime = pd.to_datetime(starttime)
        endtime = pd.to_datetime(endtime)
        # Check if the given time is within the range
        return ((searchdf >= starttime) & (searchdf <= endtime)).any()
    
    for i in range(len(hilotides)):
        searhDT = hilotides.loc[i, 'Date Time']
        boolTagOn = any(prh['DS_HHMMSS'].isin([searhDT])) # is the tag on?
        if boolTagOn == True: 
            hilotides.loc[i,'tagon'] = bool(1) 
        else: 
            hilotides.loc[i,'tagon'] = bool(0)


    for i in range(len(hilotides)-1):
        starttime = hilotides.loc[i, 'Date Time']
        endtime = hilotides.loc[i+1, 'Date Time']
        
        for i in range(len(hilotides)):
            # IF this is not the last row
            nextrow = i+1
            if i < len(hilotides)-1:
                if hilotides.loc[i, ' Type'] == 'L' and hilotides.loc[nextrow, ' Type'] == 'H':
                    hilotides.loc[i, 'risefall'] = 'R'    
                else:
                    hilotides.loc[i, 'risefall'] = 'F'
        
                periodstart = pd.to_datetime(hilotides.loc[i, 'Date Time'])
                periodend = pd.to_datetime(hilotides.loc[nextrow, 'Date Time'])
                hilotides.loc[i,'periodStartDS'] = periodstart
                hilotides.loc[i,'periodStopDS'] = periodend
                hilotides.loc[i, 'boolFeed'] = isinTimeRange(feed['sFEDS'], periodstart, periodend)
                # FIND THE CENTER TIME TO MAKE YOUR QUARTERS
                timediff = periodend-periodstart
                hilotides.loc[i, 'centerPeriod'] = pd.to_datetime(periodstart+timediff/2)
                
                # if the tag was put on after the start of the period and before the end
                if tagonDS > periodstart and tagonDS<periodend:
                    periodstart = tagonDS
                    print('changing start time to tag on')
                else:
                    print('')
                    
                # if the tag came off after the start of the period and before the end
                if tagoffDS > periodstart and tagoffDS < periodend:
                    print('changing end time to tag off')
                    periodend = tagoffDS
                else:
                    print('')
                    
                timediff = periodend-periodstart
                hilotides.loc[i, 'durPeriod'] = timediff
                hilotides.loc[i, 'boolFeed'] = isinTimeRange(feed['sFEDS'], periodstart, periodend)
                # if the tag came off before the center of the period, find the duration and set the second quarter to nan
                if periodend<=hilotides.loc[i, 'centerPeriod']:
                    hilotides.loc[i, 'durQ1'] = periodend-periodstart 
                    hilotides.loc[i, 'durQ2'] = np.nan
                # if the tag came off after the center period, 
                else: 
                    hilotides.loc[i, 'durQ1'] = hilotides.loc[i, 'centerPeriod'] - hilotides.loc[i,'periodStartDS'] 
                    hilotides.loc[i, 'durQ2'] = periodend - hilotides.loc[i, 'centerPeriod'] 
                    
            
            else: 
                starttime = hilotides.loc[i, 'Date Time']
                endtime = np.nan
                hilotides.loc[i,'periodStartDS'] = starttime
                hilotides.loc[i,'periodStopDS'] = endtime
                hilotides.loc[i, 'boolFeed'] = False
                hilotides.loc[i, 'risefall'] = np.nan
                hilotides.loc[i, 'centerPeriod'] = np.nan
                hilotides.loc[i, 'durPeriod'] = np.nan
                hilotides.loc[i, 'durQ1'] = np.nan
                hilotides.loc[i, 'durQ2'] = np.nan     
                
        hilotides = hilotides[~((hilotides['tagon'] == False) & (hilotides['tagon'].shift(-1) == False))].reset_index(drop = True)

        hilotides['numTPeriod'] = (hilotides.index)
        if hilotides.loc[0, 'numTPeriod'] == 0:
            hilotides['numTPeriod'] = (hilotides.index)+1
        else:
            hilotides['numTPeriod'] = (hilotides.index)

        for i in range(len(depTides)):
            currentTime = pd.to_datetime(depTides.loc[i, 'DateTime'])
            diff = currentTime - hilotides['Date Time']
            indHiLoTides = diff[diff > pd.Timedelta(0)].idxmin()
            depTides.loc[i, 'numTPeriod'] = hilotides.loc[indHiLoTides, 'numTPeriod']
            depTides.loc[i, 'durPeriod'] = hilotides.loc[indHiLoTides, 'durPeriod']
        
            depTides.loc[i, 'risefall'] = hilotides.loc[indHiLoTides, 'risefall']
            centerPeriod = hilotides.loc[indHiLoTides, 'centerPeriod']
            thisPeriod = str(int(depTides.loc[i, 'numTPeriod']))
            
            if currentTime <= centerPeriod:
                #thisPeriod = str(int(depTides.loc[i, 'numTPeriod']))
                depTides.loc[i, 'numTQuarter'] = str(1)
                depTides.loc[i, 'durQuarter'] = hilotides.loc[indHiLoTides, 'durQ1']
                depTides.loc[i, 'descTide'] = f"P{thisPeriod}{depTides.loc[i, 'risefall']}Q{depTides.loc[i, 'numTQuarter']}"    
            else:
                depTides.loc[i, 'numTQuarter'] = str(2)
                depTides.loc[i, 'durQuarter'] = hilotides.loc[indHiLoTides, 'durQ2']
                depTides.loc[i, 'descTide'] = f"P{thisPeriod}{depTides.loc[i, 'risefall']}Q{depTides.loc[i, 'numTQuarter']}"  
        
        for i in range(len(feed)):
        # get the tidal height for this feeding event and the tidal decription
            currentTime = pd.to_datetime(feed.loc[i, 'sFEDS'])
            inddepTides = (depTides['DateTime'] - currentTime).abs().idxmin()
            closestTime = depTides.loc[inddepTides, 'DateTime']
            feed.loc[i, 'tidalheight_ft'] = depTides.loc[inddepTides, 'Prediction']
            feed.loc[i, 'descTide'] = depTides.loc[inddepTides, 'descTide']
            feed.loc[i, 'risefall'] = depTides.loc[inddepTides, 'risefall']
        
            feed.loc[i, 'numTPeriod'] = depTides.loc[inddepTides, 'numTPeriod']
            feed.loc[i, 'durPeriod'] = depTides.loc[inddepTides, 'durPeriod']
            feed.loc[i, 'durQuarter'] = depTides.loc[inddepTides, 'durQuarter']
        return(hilotides, depTides, feed)

